<a href="https://colab.research.google.com/github/AlessandroEspinosa2003/FishData/blob/SeanBranch/DataProcess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import pandas as pd
import flask

# Load the data set from the special file.
data = np.load('FishData.txt.npy')

# Test: Did everything load?
data

# Separate the data by species. Each new array should only contain data
# about the particular species.
species = ['CRAB, BLUE', 'TUNA, YELLOWFIN', 'SNAPPER, RED', 'SHRIMP, ROCK',
           'MACKAREL, KING']

# These return the rows containing the indicated species.
crab = np.where(data[:, 2] == species[0])
tuna = np.where(data[:, 2] == species[1])
snap = np.where(data[:, 2] == species[2])
shrimp = np.where(data[:, 2] == species[3])
mack = np.where(data[:, 2] == species[4])

#